# Goal : Pose Detection with OpenPose

This notebook uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) to detect/track multi person poses on a given video.

For other deep-learning Colab notebooks, visit [tugstugi/dl-colab-notebooks](https://github.com/tugstugi/dl-colab-notebooks).


## 1. Google drive 연결

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

%cd /content
!mkdir input
!wget https://github.com/jeeenn/SportsFriends/raw/master/Data/yogaFriend_demo.mp4 -P /content/input

## 2. Install OpenPose

In [2]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz

In [ ]:
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev


In [ ]:
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

## 3. Detect poses on a test video

We are going to detect poses on the following yoga video:

*   set input/output/model path 
*   set input file type/input_cut_time

In [53]:
import glob
import os
from google.colab import files

model_path = '/content/openpose'  
#input_path = '/content/drive/My Drive/yoga_video'
input_path = '/content/input'
output_path = '/content/output'
output_file_path = "".join([output_path, '/file'])
output_json_path = "".join([output_path, '/json'])

os.system(f"rm -rf '{output_path}'")
os.mkdir(output_path)
os.mkdir(output_file_path)
os.mkdir(output_json_path)

input_file_type = "mp4"
input_cut_time = 10

%cd $model_path

/content/openpose


*   *file cut /detecte key point /output : video file & json*
*   origin_file_path = 'content/output/Abhay_Bhuj.mp4' <br/>
    origin_file_name = 'Abhay_Bhuj.mp4' <br/>
    temp_file_name = 'content/output/openpose.avi' <br/>
*   결과    
    output_file_name = 'content/output/file/Abhay_Bhuj.mp4' <br/>
    output_json_name = 'content/output/json/Abhay_Bhuj_xxx.json'

In [49]:
def detect_keypoint_file(file_name):
  
  origin_file_path = "".join([output_path, '/', file_name.split('/')[-1]])
  origin_file_name = file_name.split('/')[-1].split('.')[0]  
  temp_file_name = "".join([output_path, '/', 'openpose.avi'])
  output_file_name = "".join([output_file_path, '/', origin_file_name, '.', input_file_type])
  output_json_name = "".join([output_json_path, '/', origin_file_name])
    
  os.system(f"ffmpeg -y -loglevel info -i '{file_name}' -t {input_cut_time} '{origin_file_path}'")
  os.system(f"./build/examples/openpose/openpose.bin --keypoint_scale 3 --frame_step 3 --video '{origin_file_path}' --write_json '{output_json_name}/' --display 0 --write_video '{temp_file_name}'")
  os.system(f"ffmpeg -y -loglevel info -i '{temp_file_name}' '{output_file_name}'")
  os.system(f"rm -rf '{origin_file_path}'")
  os.system(f"rm -rf '{temp_file_name}'")
  

In [57]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

* file(video).zip / json.zip 생성
* local machine download

In [7]:
def download_file():
  file_zip_name = "".join([output_file_path, '.zip'])
  json_zip_name = "".join([output_json_path, '.zip'])

  os.system(f"zip -s 25 '{file_zip_name}' '{output_file_path}'")
  files.download(file_zip_name)
  
  os.system(f"zip -s 25 '{json_zip_name}' '{output_json_path}'")
  files.download(json_zip_name)  

*   loop input file count

In [ ]:
for file_name in glob.glob("".join([input_path, '/*', input_file_type])):     
  detect_keypoint_file(file_name)  

In [ ]:
show_local_mp4_video('/content/output/file/yogaFriend_demo.mp4', width=480, height=360)

In [ ]:
download_file()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>